In [1]:
import pdfplumber
import requests
import pandas as pd

**1 - Data Prepocessing** 
<br> Preliminary data cleaning and inspection before reformatting as SQL tables
<br> Involves: lowering all string, making units consistent, making data formats consistent, dropping unnecessary data, merging column data 

In [13]:
# Target data spread = 2013 - 2021, across all UK main waterways
# greenpeace labs technical report - pages 16 to 17 - PDF - Feb/March 2019 - All UK
import re

with pdfplumber.open("data/GP_mps.pdf") as pdf:
    page = pdf.pages[15]
    first_page_dat = pd.Series(page.extract_table(table_settings = {"horizontal_strategy": "text"}))
    
    sec_page = pdf.pages[16]
    second_page_dat = pd.Series(sec_page.extract_table(table_settings = {"horizontal_strategy": "text"}))
    
    page_dat = pd.concat([first_page_dat, second_page_dat])
    #im = page.to_image(resolution = 120)
    #im.show()
print(page_dat)

0     [, None, None, , None, None, , None, None, , N...
1     [, None, None, , None, None, , None, None, , T...
2     [River, None, None, Sampling location, None, N...
3     [, None, None, , None, None, , None, None, flo...
4     [, Exe, , , Footbridge over the Exe in central...
                            ...                        
27     [, , , , , , , , , , , , , , , , , , , , , , , ]
28    [, Lagan, , , , , , 28/02/2019, , , 6.38, , , ...
29    [, None, None, , below sewage outlet, , , None...
30    [, , , , , , , , , , , None, None, None, None,...
31    [, Lagan, , , Above Stranmillis Weir (NTL) in ...
Length: 95, dtype: object


In [14]:
page_dat.drop([0,1,2,3], inplace = True)

In [15]:
# Transform pdf extraction into a dataframe
# Record format: Source, Total plastic Mass (per km river flow), Microplastics (per km river flow), Sampling Dates, Sampling Location, Sampling Method
print(page_dat)
gp_dat = pd.DataFrame((i for i in page_dat), columns = ("drop 6", "samp_location", "drop1", "drop 5", "drop 18", "drop 19", "drop 20", "samp_date", "drop 7", "drop 8", "drop2", "drop 9", "drop 10", "drop3", "drop 11", "drop 12", "drop4", "drop 13", "drop 14", "sum_km2", "drop 15", "drop 16", "sum_micro_km2", "drop 17"))

4     [, Exe, , , Footbridge over the Exe in central...
5     [, None, None, , None, None, , None, None, , N...
6     [, Exe, , , Footbridge over the Exe in central...
7     [, None, None, , None, None, , None, None, , N...
8     [, Exe, , , Footbridge over the Exe in central...
                            ...                        
27     [, , , , , , , , , , , , , , , , , , , , , , , ]
28    [, Lagan, , , , , , 28/02/2019, , , 6.38, , , ...
29    [, None, None, , below sewage outlet, , , None...
30    [, , , , , , , , , , , None, None, None, None,...
31    [, Lagan, , , Above Stranmillis Weir (NTL) in ...
Length: 87, dtype: object


In [16]:
gp_dat = gp_dat.drop(columns = [col for col in gp_dat.columns if "drop" in col])
gp_dat = gp_dat.dropna()
gp_dat.reset_index(inplace = True, drop = True)

In [17]:
# Add source and sampling method details, format datetime
gp_dat["source"] = "https://www.greenpeace.to/greenpeace/wp-content/uploads/2019/06/GRL-TR-04-2019-plastics-in-UK-rivers.pdf"
gp_dat["samp_method"] = "Net Tow sampling"

gp_dat["samp_date"] = pd.to_datetime(gp_dat["samp_date"], format = '%d/%m/%Y', dayfirst = True)
gp_dat = gp_dat.reindex(columns = ["source", "samp_location", "samp_date", "sum_micro_km2", "sum_km2", "samp_method"])

gp_dat.head(n = 10)

,source,samp_location,samp_date,sum_micro_km2,sum_km2,samp_method
0,https://www.greenpeace.to/greenpeace/wp-conten...,Exe,2019-02-06,4821,11571,Net Tow sampling
1,https://www.greenpeace.to/greenpeace/wp-conten...,Thames,2019-02-11,52926,75938,Net Tow sampling
2,https://www.greenpeace.to/greenpeace/wp-conten...,Thames,2019-03-12,38233,61761,Net Tow sampling
3,https://www.greenpeace.to/greenpeace/wp-conten...,Thames,2019-03-12,14205,20518,Net Tow sampling
4,https://www.greenpeace.to/greenpeace/wp-conten...,Thames,2019-03-12,5090,10181,Net Tow sampling
5,https://www.greenpeace.to/greenpeace/wp-conten...,Severn,2019-02-19,12353,25940,Net Tow sampling
6,https://www.greenpeace.to/greenpeace/wp-conten...,Severn,2019-03-04,11244,33731,Net Tow sampling
7,https://www.greenpeace.to/greenpeace/wp-conten...,,NaT,,,Net Tow sampling
8,https://www.greenpeace.to/greenpeace/wp-conten...,,NaT,,,Net Tow sampling
9,https://www.greenpeace.to/greenpeace/wp-conten...,Severn,2019-03-04,0,12833,Net Tow sampling


In [18]:
# england .gov Microplastics cefas Data - CSV - 2013 to 2021 - England & Wales
cefas_dat = pd.read_csv("data/cefas_mps.csv")
cefas_dat.drop(["Rep", "Station", "Latitude", "Longitude", "Sediment dried weight in g","Gravel %", "Sand %", "Silt %", "OC2", "OC63", "Nit63", "Nit2"], axis = 1, inplace = True)
cefas_dat.head()

,Year,Number of particles,Number of particles Blank_Corrected,Number of particles per kg d.w. sediment,Coastal,Region
0,2013,41,38,7600,coastal,Northern North Sea
1,2013,61,58,11600,coastal,Northern North Sea
2,2013,45,42,8400,coastal,Northern North Sea
3,2013,12,9,1800,coastal,Northern North Sea
4,2013,20,17,3400,coastal,Northern North Sea


In [19]:
# scotland .gov Microplastics in Scotlands Seas - CSV - 2013 to Present - Scotland
scots_dat = pd.read_csv("data/Scotland_mps.csv")
scots_dat.drop(["Cruise", "Season", "Field_ID", "TOW_ID", "Year", "Decimal Latitude at deployment (WGS84)", "Decimal Longitude at deployment (WGS84)", "Tow_length (m)", "vol_filtered_m3", "WIND_SPEED", "WIND_Direction", "SEA_STATE", "Ship Speed"], axis = 1, inplace = True)
# now drop columns with Retrieve, km2, LTE, GT5
scots_dat.head()


,Region,DATE_Deploy,TIME_Deploy,FLOW_METER_READING_deploy,DATE_Retrieve,TIME_Retrieve,decLat_Retrieve,decLong_Retrieve,FLOW_METER_READING_retrieve,SUM_micro_km2,...,film_LTE5mm_count,nurdles_LTE5mm_count,paint_flakes_LTE5mm_count,fibres_GT5mm_count,beads_GT5mm_count,polystyrene_GT5mm_count,fragments_GT5mm_count,film_GT5mm_count,nurdles_GT5mm_count,paint_flakes_GT5mm_count
0,Forth and Tay,06/01/2014,16:23,8212,06/01/2014,17:05,56.08633,-2.83967,24662,5715,...,0,1,0,0,0,0,0,0,0,0
1,Forth and Tay,06/01/2014,17:24,24662,06/01/2014,18:25,56.13600,-2.69017,39113,9463,...,1,1,5,0,0,0,0,0,0,0
2,Moray Firth,08/01/2014,14:01,46519,08/01/2014,15:05,57.84333,-3.73500,79946,0,...,0,0,0,0,0,0,0,0,0,0
3,Moray Firth,08/01/2014,15:40,79946,08/01/2014,16:10,57.88667,-3.84950,300,0,...,0,0,0,0,0,0,0,0,0,0
4,Moray Firth,08/01/2014,18:50,300,08/01/2014,19:50,57.94650,-3.28083,33753,1022,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# Normalise all collected data - date format, all lowercase, river/location names, units 
def normal_dat(data):
    """
    This function normalises an input DataFrame, ready for input into a larger microplastics dataset.
    
    Parameters:
    data (DataFrame): Input dataset
    
    Returns:
    norm_data (DataFrame): Normalised dataset
    """
    # lower case 